# 1. Setup and Get Data

### 1.1 Install Dependencies and Setup

In [ ]:
%pip install --user labelme tensorflow tensorflow-gpu opencv-python matplotlib albumentations split-folders imutils scikit-learn keras

# 2. Review Dataset and Build Image Loading Function

### 2.1 Import Tensorflow and Dependences

In [4]:
import os
import cv2
import json

import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from scipy.io import loadmat
import tqdm


### 2.2 Limit GPU Memory Growth

In [5]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [6]:
tf.config.list_physical_devices('GPU')

[]

### 2.3 Load Image into TF Data Pipeline

In [13]:
import os
import torch
import scipy.io
import traceback
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import torchvision.transforms as transforms

from models.config import rfcn_config as cfg

from torch.utils.data import Dataset, DataLoader
from PIL import Image


# https://www.cs.virginia.edu/~vicente/recognition/notebooks/image_processing_lab.html
class WiderFaceDataset(Dataset):

    def __init__(self, image_path, metadata_path, object_min_dim=300, transform=None):
        self.image_path = image_path
        self.metadata_path = metadata_path
        self.transform = transform
        self.pil2tensor = transforms.ToTensor()

        self.convert_to_image_list(self.metadata_path, object_min_dim)

    def convert_to_image_list(self, path, object_min_dim):
        self.f = scipy.io.loadmat(path)
        self.event_list = self.f.get('event_list')
        self.file_list = self.f.get('file_list')
        self.face_bbx_list = self.f.get('face_bbx_list')
        self.occlusion_label_list = self.f.get('occlusion_label_list')
        self.pose_label_list = self.f.get('pose_label_list')

        image_metadata = {
            'image_location': [],
            'image_ground_truth': []
        }

        for idx, event in enumerate(self.event_list):
            event = event[0][0]

            for file_idx, file_path in enumerate(self.file_list[idx][0]):
                file_name = file_path[0][0] + '.jpg'
                file_name = event + '/' + file_name
                file_path = os.path.abspath(self.image_path + '/' + file_name)
                
                bounding_boxes = self.face_bbx_list[idx][0][file_idx][0]
                occlusions = self.occlusion_label_list[idx][0][file_idx][0]
                pose = self.pose_label_list[idx][0][file_idx][0]

                # Filter out medium and hard bounding_boxes if any 
                bounding_boxes_filtered = []
                for occlusion_idx, occlusion in enumerate(occlusions):
                    if occlusion[0] == 0 and pose[occlusion_idx][0] == 0:
                        # Now, get only large Bboxes
                        #print(bounding_boxes[occlusion_idx])
                        if bounding_boxes[occlusion_idx][2] > object_min_dim and bounding_boxes[occlusion_idx][3] > object_min_dim:
                            bounding_boxes_filtered.append(bounding_boxes[occlusion_idx])
                
                
                if len(bounding_boxes_filtered) > 0:
                    image_metadata['image_location'].append(file_path)
                    image_metadata['image_ground_truth'].append(bounding_boxes_filtered)

                # Plot anchors
                # positive_anchors, negative_anchors = get_regions(np.array(Image.open(file_path).convert('RGB'), dtype=np.uint32), 100, bounding_boxes_filtered)
                # print("positive:",len(positive_anchors))
                # print("negative:",len(negative_anchors))
                # self.plot_boxes(file_path, positive_anchors, [], bounding_boxes_filtered)

                # break
            # break

        # convert to pandas
        self.dataset = pd.DataFrame.from_dict(image_metadata)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        try:
            with open(self.dataset.iloc[idx]['image_location'], 'rb') as f:
            #with open("/home/vedvalsangkar/Face-R-FCN/data/widerface/WIDER_train/images/0--Parade/0_Parade_Parade_0_452.jpg", 'rb') as f:
                image = Image.open(f)
                image = image.convert('RGB')
                #print(image.size)
                image, boxes = self.resize_image(image, self.dataset.iloc[idx]['image_ground_truth'],
                                                 dimension=cfg.IMAGE_INPUT_DIMS)

        except Exception:
            print("Image not found..", traceback.format_exception())
            return ([], self.dataset.iloc[idx]['image_ground_truth'])

        image_tensor = self.pil2tensor(image)
        # ground_truth_tensor = boxes
        ground_truth_tensor = torch.tensor(np.array(boxes))

        return (image_tensor, ground_truth_tensor, self.dataset.iloc[idx]['image_location'])

    # Referenced from: https://jdhao.github.io/2017/11/06/resize-image-to-square-with-padding/
    def resize_image(self, im, b_boxes, dimension=cfg.IMAGE_INPUT_DIMS):
        b_boxes = np.array(b_boxes, dtype=np.float)

        old_size = im.size
        
        ratio = float(dimension) / max(old_size)
        new_size = tuple([int(x * ratio) for x in old_size])

        im = im.resize(new_size, Image.ANTIALIAS)

        offset_x = (dimension - new_size[0]) // 2
        offset_y = (dimension - new_size[1]) // 2

        # create a new image and paste the resized on it
        new_im = Image.new("RGB", (dimension, dimension))
        new_im.paste(im, (offset_x, offset_y))

        # Re-size and offset bounding boxes based on image
        b_boxes[:,0] = (b_boxes[:,0] * ratio) + offset_x
        b_boxes[:,1] = (b_boxes[:,1] * ratio) + offset_y
        
        b_boxes[:,0] = np.clip(b_boxes[:,0], 0, None)
        b_boxes[:,1] = np.clip(b_boxes[:,1], 0, None)

        b_boxes[:,2] = np.clip(b_boxes[:,2],1, None) * ratio
        b_boxes[:,3] = np.clip(b_boxes[:,3],1, None) * ratio

        """for box in b_boxes:
            x = int(abs(box[0] * ratio + offset_x))
            y = int(abs(box[1] * ratio + offset_y))
            l = int(box[2] * ratio)
            b = int(box[3] * ratio)
            new_box = [x, y, l, b]
            results.append(new_box)
        """
        return new_im, b_boxes[:60,:]

    def plot_boxes(self, file, positive_anchors, negative_anchors, boxes):
        im = np.array(Image.open(file).convert('RGB'), dtype=np.uint8)

        # Create figure and axes
        fig, ax = plt.subplots(1)

        # Display the image
        ax.imshow(im)

        if positive_anchors:
            for i, box in enumerate(positive_anchors):
                rect = patches.Rectangle((box[0], box[1]), box[2], box[3], linewidth=1, edgecolor='green',
                                         facecolor='none')
                ax.add_patch(rect)

        if negative_anchors:
            for i, box in enumerate(negative_anchors):
                rect = patches.Rectangle((box[0], box[1]), box[2], box[3], linewidth=1, edgecolor='red',
                                         facecolor='none')
                ax.add_patch(rect)

        for i, box in enumerate(boxes):
            rect = patches.Rectangle((box[0], box[1]), box[2], box[3], linewidth=1, edgecolor='blue', facecolor='none')
            ax.add_patch(rect)

        plt.show()


def get_regions(features, N, list_bb):
    """
    Function to generate Positive and Negative anchors.
    :param features:  Extracted features
    :param N:         Number of bounding boxes
    :param list_bb:   Bounding boxes (scaled to current configuration)
    :return:          Positive and negative anchors.
    """

    # box_size = [(32, 32), (64, 32), (32, 64)]
    box_size = [(128, 128), (256, 128), (128, 256), (256, 256), (256, 512), (512, 256)]
    """Sizes for region proposals"""

    feat_h, feat_w, _ = features.shape

    print("Shape of features:", features.shape)
    print("Shape of bounding box:", list_bb[0])

    pos_anc = []
    neg_anc = []
    scale = 1

    for bs in box_size:
        for i in range(0, feat_h):
            for j in range(0, feat_w):

                max_iou = 0

                x = min(feat_h, max(0, (i - (bs[0] // 2))))
                y = min(feat_w, max(0, (j - (bs[1] // 2))))

                # im_slice = features[x:x + bs[0], y:y + bs[1]]
                frame_a = (x, y, x + bs[0], y + bs[1])
                """
                Frame A is the sliding window for calculation.
                """

                for bb in list_bb:
                    frame_b = (bb[1], bb[0], bb[1] + bb[3], bb[0] + bb[2])
                    """
                    Frame B is the input """
                    iou = calc_IOU(frame_a, frame_b)
                    if max_iou < iou:
                        max_iou = iou

                    # print(max_iou)

                if max_iou > 0.762:
                    print(frame_a, frame_b, max_iou)
                    # pos_anc.append((x * scale, (x + bs[0]) * scale, y * scale, (y + bs[1]) * scale))
                    pos_anc.append([y * scale, x * scale, bs[1], bs[0]])
                elif max_iou < 0.05:
                    # neg_anc.append((x * scale, (x + bs[0]) * scale, y * scale, (y + bs[1]) * scale))
                    neg_anc.append([y * scale, x * scale, bs[1], bs[0]])

    return pos_anc, neg_anc


def calc_IOU(boxA, boxB):
    """
    Function taken from this site:
    https://www.pyimagesearch.com/2016/11/07/intersection-over-union-iou-for-object-detection/
    :param boxA: (X_min, Y_min, X_max, Y_max)
    :param boxB: (X_min, Y_min, X_max, Y_max)
    :return: IOU area ratio.
    """
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)

    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)


FileNotFoundError: [WinError 3] The system cannot find the path specified: '0--Parade/0_Parade_marchingband_1_849.jpg'

In [ ]:
images = tf.data.Dataset.list_files('data\\img\\img\\*.jpg')

In [ ]:
train_images.as_numpy_iterator().next()

In [ ]:
def load_image(x): 
    byte_img = tf.io.read_file(x)
    img = tf.io.decode_jpeg(byte_img)
    return img

In [ ]:
train_images = train_images.map(load_image)

In [ ]:
train_images.as_numpy_iterator().next()

In [ ]:
type(train_images)

### 2.4 View Raw Images with Matplotlib

In [ ]:
image_generator = train_images.batch(4).as_numpy_iterator()

In [ ]:
plot_images = next(image_generator)

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, image in enumerate(plot_images):
    ax[idx].imshow(image) 
plt.show()

# 3. Partition Unargumented Data

### 3.1 Split data into training and testing and valid data

In [ ]:
import splitfolders
splitfolders.ratio('./data/img', output="./data/ttvimg", seed=1337, ratio=(.8, 0.1,0.1))

### 3.2 Move the Matching Labels

In [ ]:
for folder in ['train','test','val']:
    for file in os.listdir(os.path.join('data', folder, 'images')):
        
        filename = file.split('.')[0]+'.json'
        existing_filepath = os.path.join('data','labels', filename)
        if os.path.exists(existing_filepath): 
            new_filepath = os.path.join('data',folder,'labels',filename)
            os.replace(existing_filepath, new_filepath)      

# 4. Setup Albumentations

### 4.1 Setup Albumentations Transform Pipeline

In [ ]:
import albumentations as alb

In [ ]:
augmentor = alb.Compose([alb.RandomCrop(width=1280, height=720)], bbox_params=alb.BboxParams(format='albumentations', label_fields=['class_labels']))

### 4.2 Load a Test Image and Annotation with OpenCV and JSON

In [ ]:
testImage = cv2.imread(os.path.join('data', 'WIDERFace', 'train','0--Parade','0_Parade_marchingband_1_5.jpg'))

In [ ]:
with open(os.path.join('data', 'train', 'labels', '1.json'), 'r') as f:
    label = json.load(f)

In [ ]:
label['shapes'][1]['points']

### 4.3 Extract Coordinates and Rescale to Match Image Resolution

In [ ]:
coords = [0,0,0,0]
coords[0] = label['shapes'][1]['points'][0][0]
coords[1] = label['shapes'][1]['points'][0][1]
coords[2] = label['shapes'][1]['points'][1][0]
coords[3] = label['shapes'][1]['points'][1][1]

In [ ]:
coords

In [ ]:
coords = list(np.divide(coords, [1920,1080,1920,1080]))

In [ ]:
coords

### 4.4 Apply Augmentations and View Results

In [ ]:
augmented = augmentor(image=testImage, bboxes=[coords], class_labels=['face'])

In [ ]:
augmented['bboxes'][0][2:]

In [ ]:
augmented['bboxes']

In [ ]:
cv2.rectangle(augmented['image'], 
              tuple(np.multiply(augmented['bboxes'][0][:2], [1280, 720]).astype(int)),
              tuple(np.multiply(augmented['bboxes'][0][2:], [1280, 720]).astype(int)), 
                    (255,0,0), 2)

plt.imshow(augmented['image'])

# 5. Build and Run Augmentation Pipeline

### 5.1 Run Augmentation Pipeline

In [ ]:
for partition in ['train']: 
    for image in os.listdir(os.path.join('data', partition, 'images')):
        img = cv2.imread(os.path.join('data', partition, 'images', image))

        coords = [0,0,0.00001,0.00001]
        label_path = os.path.join('data', partition, 'labels', f'{image.split(".")[0]}.json')
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                label = json.load(f)

            coords[0] = label['shapes'][0]['points'][0][0]
            coords[1] = label['shapes'][0]['points'][0][1]
            coords[2] = label['shapes'][0]['points'][1][0]
            coords[3] = label['shapes'][0]['points'][1][1]
            coords = list(np.divide(coords, [1920,1080,1920,1080]))

        try: 
            for x in range(60):
                augmented = augmentor(image=img, bboxes=[coords], class_labels=['face'])
                cv2.imwrite(os.path.join('augmented-data', partition, 'images', f'{image.split(".")[0]}.{x}.jpg'), augmented['image'])

                annotation = {}
                annotation['image'] = image

                if os.path.exists(label_path):
                    if len(augmented['bboxes']) == 0: 
                        annotation['bbox'] = [0,0,0,0]
                        annotation['class'] = 0 
                    else: 
                        annotation['bbox'] = augmented['bboxes'][0]
                        annotation['class'] = 1
                else: 
                    annotation['bbox'] = [0,0,0,0]
                    annotation['class'] = 0 


                with open(os.path.join('augmented-data', partition, 'labels', f'{image.split(".")[0]}.{x}.json'), 'w') as f:
                    json.dump(annotation, f)

        except Exception as e:
            print(e)

### 5.2 Load Augmented Images to Tensorflow Dataset

In [ ]:
train_images = tf.data.Dataset.list_files('augmented-data\\train\\images\\*.jpg', shuffle=False)
train_images = train_images.map(load_image)
train_images = train_images.map(lambda x: tf.image.resize(x, (240,240)))
train_images = train_images.map(lambda x: x/255)

In [ ]:
test_images = tf.data.Dataset.list_files('augmented-data\\test\\images\\*.jpg', shuffle=False)
test_images = test_images.map(load_image)
test_images = test_images.map(lambda x: tf.image.resize(x, (240,240)))
test_images = test_images.map(lambda x: x/255)

In [ ]:
val_images = tf.data.Dataset.list_files('augmented-data\\val\\images\\*.jpg', shuffle=False)
val_images = val_images.map(load_image)
val_images = val_images.map(lambda x: tf.image.resize(x, (240,240)))
val_images = val_images.map(lambda x: x/255)

In [ ]:
train_images.as_numpy_iterator().next()

# 6. Prepare Labels

### 6.1 Build Label Loading Function

In [ ]:
def load_labels(label_path):
    with open(label_path.numpy(), 'r', encoding = "utf-8") as f:
        label = json.load(f)
        
    return [label['class']], label['bbox']

### 5.2 Load Labels to Tensorflow Dataset

In [ ]:
train_labels = tf.data.Dataset.list_files('augmented-data\\train\\labels\\*.json', shuffle=False)
train_labels = train_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))

# Load the annotations file
train_annotations_path = 'data\\wider_face_split\\wider_face_train_bbx_gt.txt'
train_annotations = []
with open(train_annotations_path, 'r') as file:
    for line in file:
        train_annotations.append(list(map(int, line.strip().split())))

In [ ]:
# Load the annotations file
train_annotations_path = 'data\\wider_face_split\\wider_face_train_bbx_gt.txt'
train_annotations = []
with open(train_annotations_path, 'r') as file:
    for line in file:
        train_annotations.append(list(map(int, line.strip().split())))

In [ ]:
# Load the annotations file
val_annotations_path = 'data\\wider_face_split\\wider_face_train_bbx_gt.txt'
val_annotations = []
with open(val_annotations_path, 'r') as file:
    for line in file:
        val_annotations.append(list(map(int, line.strip().split())))

In [ ]:
train_annotations.as_numpy_iterator().next()

# 6. Combine Labels and Image Samples

### 6.1 Check Partition Length

In [ ]:
len(train_images), len(test_images), len(val_images), len(train_annotations), len(test_annotations), len(val_annotations)

### 6.2 Final Datasets (Images/Labels)

In [ ]:
train = tf.data.Dataset.zip((train_images, train_labels))
train = train.shuffle(5000)
train = train.batch(8)
train = train.prefetch(4)

In [ ]:
test = tf.data.Dataset.zip((test_images, test_labels))
test = test.shuffle(1300)
test = test.batch(8)
test = test.prefetch(4)

In [ ]:
val = tf.data.Dataset.zip((val_images, val_labels))
val = val.shuffle(1000)
val = val.batch(8)
val = val.prefetch(4)

In [ ]:
train.as_numpy_iterator().next()

### 6.3 View Images and Annotations

In [ ]:
data_samples = train.as_numpy_iterator()

In [ ]:
res = data_samples.next()

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(40,40))
for idx in range(4): 
    sample_image = res[0][idx]
    sample_coords = res[1][1][idx]
    
    cv2.rectangle(sample_image, 
                  tuple(np.multiply(sample_coords[:2], [240,240]).astype(int)),
                  tuple(np.multiply(sample_coords[2:], [240,240]).astype(int)), 
                        (255,0,0), 2)

    ax[idx].imshow(sample_image)

# 7 Build Deep Learning Model using the Functional API

### 8.1 Import Layers and Base Network

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dense, GlobalMaxPooling2D
from tensorflow.keras.applications import VGG16

### 7.2 Download VGG16

In [ ]:
vgg = VGG16(include_top=False)

In [ ]:
vgg.summary()

### 7.3 Build Instance of Network

In [ ]:
def build_model(): 
    input_layer = Input(shape=(240,240,3))
    
    vgg = VGG16(include_top=False)(input_layer)

    # Classification Model  
    f1 = GlobalMaxPooling2D()(vgg)
    class1 = Dense(2048, activation='relu')(f1)
    class2 = Dense(1, activation='sigmoid')(class1)
    
    # Bounding box model
    f2 = GlobalMaxPooling2D()(vgg)
    regress1 = Dense(2048, activation='relu')(f2)
    regress2 = Dense(4, activation='sigmoid')(regress1)
    
    facedetector = Model(inputs=input_layer, outputs=[class2, regress2])
    return facedetector

### 7.4 Test out Neural Network

In [ ]:
facedetector = build_model()

In [ ]:
facedetector.summary()

In [ ]:
X, y = train.as_numpy_iterator().next()

In [ ]:
X.shape

In [ ]:
classes, coords = facedetector.predict(X)

In [ ]:
classes, coords

# 8 Define Losses and Optimisers

### 8.1 Define Optimiser and LR

In [ ]:
batches_per_epoch = len(train)
lr_decay = (1./0.75-1)/batches_per_epoch

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0001, decay=lr_decay)

### 8.2 Create Localisation Loss and Classification Loss

In [ ]:
def localisation_loss(y_true, y_pred):
    delta_coord = tf.reduce_sum(tf.square(y_true[:,:2] - y_pred[:,:2]))

    h_true = y_true[:,3] - y_true[:,1]
    w_true = y_true[:,2] - y_true[:,0]

    h_pred = y_pred[:,3] - y_pred[:,1]
    w_pred = y_pred[:,2] - y_pred[:,0]

    delta_size = tf.reduce_sum(tf.square(w_true - w_pred) + tf.square(h_true - h_pred))

    return delta_coord + delta_size

In [ ]:
classloss = tf.keras.losses.BinaryCrossentropy()
regressloss = localisation_loss

### 7.3 Test out Loss Metrics

In [ ]:
localisation_loss(y[1], coords)

In [ ]:
classloss(y[0], classes)

In [ ]:
regressloss(y[1], coords)

# 8 Train Neural Network

### 9.1 Create Custom Model Class

In [ ]:
class FaceTracker(Model):
    def __init__(self, facecctv, **kwargs):
        super().__init__(**kwargs)
        self.model = facecctv

    def compile(self, classloss, localisation_loss, opt, **kwargs):
        super().compile(**kwargs)
        self.clsloss = classloss
        self.localloss = localisation_loss
        self.opt = opt

    def train_step(self, batch, **kwargs):

        X, y = batch

        with tf.GradientTape() as tape:
            classes, coords = self.model(X, training=True)

            batch_class_loss = self.clsloss(y[0], classes)
            batch_localisation_loss = self.localloss(tf.cast(y[1], tf.float32), coords)

            total_loss = (0.5*batch_class_loss) + batch_localisation_loss

            gradient = tape.gradient(total_loss, self.model.trainable_variables)
        
        self.opt.apply_gradients(zip(gradient, self.model.trainable_variables))

        return {"total_loss":total_loss, "class_loss":batch_class_loss, "regress_loss":batch_localisation_loss}

    def test_step(self, batch, **kwargs):
        X, y = batch

        classes, coords = self.model(X, training=False)

        batch_class_loss = self.clsloss(y[0], classes)
        batch_localisation_loss = self.localloss(tf.cast(y[1], tf.float32), coords)
        total_loss = (0.5*batch_class_loss) + batch_localisation_loss

        return {"total_loss":total_loss, "class_loss":batch_class_loss, "regress_loss":batch_localisation_loss}

    def call(self, X, **kwargs): 
        return self.model(X, **kwargs)

In [ ]:
model = FaceTracker(facedetector)

In [ ]:
model.compile(classloss, localisation_loss, opt)

### 9.2 Train

In [ ]:
logdir='logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
hist = model.fit(train, epochs=10, validation_data=val, callbacks=[tensorboard_callback])

### 9.3 Plot Performance

In [ ]:
hist.history

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(20,5))

ax[0].plot(hist.history['total_loss'], color='teal', label='loss')
ax[0].plot(hist.history['val_total_loss'], color='orange', label='val loss')
ax[0].title.set_text('Loss')
ax[0].legend()

ax[1].plot(hist.history['class_loss'], color='teal', label='class loss')
ax[1].plot(hist.history['val_class_loss'], color='orange', label='val class loss')
ax[1].title.set_text('Classification Loss')
ax[1].legend()

ax[2].plot(hist.history['regress_loss'], color='teal', label='regress loss')
ax[2].plot(hist.history['val_regress_loss'], color='orange', label='val regress loss')
ax[2].title.set_text('Regression Loss')
ax[2].legend()

plt.show()

# 10. Make Predictions

### 10.1 Make Predictions 

In [ ]:
test_data = test.as_numpy_iterator()

In [ ]:
test_sample = test_data.next()

In [ ]:
yhat = facedetector.predict(test_sample[0])

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(40,40))
for idx in range(4): 
    sample_image = test_sample[0][idx]
    sample_coords = yhat[1][idx]
    
    if yhat[0][idx] > 0.9:
        cv2.rectangle(sample_image, 
                      tuple(np.multiply(sample_coords[:2], [240,240]).astype(int)),
                      tuple(np.multiply(sample_coords[2:], [240,240]).astype(int)), 
                            (255,0,0), 2)
    
    ax[idx].imshow(sample_image)

### 10.2 Save the Model

In [ ]:
from keras.models import load_model

In [ ]:
facedetector.save('facecctv.h5')

# 12 Testing

### 12.1 Loading Models

In [ ]:
facedetector = load_model('./models/facetracker.h5')
facecctv = load_model('facecctv.h5')

### 12.2 Testing face detection using webcam live

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    _ , frame = cap.read()
    frame = frame[50:500, 50:500,:]
    
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = tf.image.resize(rgb, (120,120))
    
    yhat = facedetector.predict(np.expand_dims(resized/255,0))
    sample_coords = yhat[1][0]
    
    if yhat[0] > 0.5: 
        # Controls the main rectangle
        cv2.rectangle(frame, 
                      tuple(np.multiply(sample_coords[:2], [1280, 720]).astype(int)),
                      tuple(np.multiply(sample_coords[2:], [1280, 720]).astype(int)), 
                            (255,0,0), 2)
        # Controls the label rectangle
        cv2.rectangle(frame, 
                      tuple(np.add(np.multiply(sample_coords[:2], [1280, 720]).astype(int), 
                                    [0,-30])),
                      tuple(np.add(np.multiply(sample_coords[:2], [1280, 720]).astype(int),
                                    [80,0])), 
                            (255,0,0), -1)
        
        # Controls the text rendered
        cv2.putText(frame, 'face', tuple(np.add(np.multiply(sample_coords[:2], [1280, 720]).astype(int),
                                               [0,-5])),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
    cv2.imshow('EyeTrack', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

### 12.3 Testing using dataset static images

In [ ]:
image = cv2.imread('data\\footage\\0.jpg')
plt.imshow(image)

rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
resized = tf.image.resize(rgb, (120, 120))

yhat = facedetector.predict(np.expand_dims(resized/255,0))
sample_coords = yhat[1][0]

if yhat[0] > 0.5: 
    # Controls the main rectangle
    cv2.rectangle(image, 
                    tuple(np.multiply(sample_coords[:2], [1280, 720]).astype(int)),
                    tuple(np.multiply(sample_coords[2:], [1280, 720]).astype(int)), 
                        (255,0,0), 2)
    # Controls the label rectangle
    cv2.rectangle(image, 
                    tuple(np.add(np.multiply(sample_coords[:2], [1280, 720]).astype(int), 
                                [0,-30])),
                    tuple(np.add(np.multiply(sample_coords[:2], [1280, 720]).astype(int),
                                [80,0])), 
                        (255,0,0), -1)
    
    # Controls the text rendered
    cv2.putText(image, 'face', tuple(np.add(np.multiply(sample_coords[:2], [1280, 720]).astype(int),
                                            [0,-5])),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)

cv2.imshow('EyeTrack', image)



In [ ]:
import os
import tensorflow as tf

from tensorflow import keras
import numpy as np
import cv2

# load the saved model
model = keras.models.load_model("facecctv.h5")

# load an image into memory
image = cv2.imread("data\\wider_face_split\\WIDER_train\\images\\0--Parade\\0_Parade_marchingband_1_5.jpg")


rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
resized = tf.image.resize(rgb, (240, 240))

prediction = model.predict(np.expand_dims(resized/255,0))
sample_coords = prediction[1][0]

#for (x, y, w, h) in prediction:
      #cv2.rectangle(image, (x, y), (w, h), (0, 255, 0), 2)

# interpret the prediction
if prediction[0][0] > 0.5:

      print("Face detected")
else:
      print("No face detected")